In [1]:
import numpy as np 
import pandas as pd
# import os
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw, ImageEnhance
# from tqdm.notebook import tqdm
from tqdm import tqdm

# import cv2
# import re
# import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
device = torch.device('cpu')
# device = torch.device('cuda')
retinanet = model.resnet101(num_classes=2, device=device)
# retinanet.to(device)

num_features_in of ResidualAfterFPN : 256


In [3]:
anchor = Anchors()
img = torch.randn([4,3,320,320])
img.shape

torch.Size([4, 3, 320, 320])

In [4]:
anchors = anchor(img)

In [5]:
anchors[:,:]

tensor([[[-12., -12.,  20.,  20.],
         [ -4., -12.,  28.,  20.],
         [  4., -12.,  36.,  20.],
         ...,
         [284., 300., 316., 332.],
         [292., 300., 324., 332.],
         [300., 300., 332., 332.]]])

In [6]:
myan = anchors[:,500:510]
myan = torch.tensor([[[148.,  84., 180., 116.],
         [156.,  84., 188., 116.],
         [164.,  84., 196., 116.],
         [172.,  84., 204., 116.],
         [180.,  84., 212., 116.],
         [188.,  84., 220., 116.],
         [196.,  84., 228., 116.],
         [204.,  84., 236., 116.],
         [212.,  84., 244., 116.],
         [260.,  84., 292., 116.]]])
a = myan[0,:,:]

In [7]:
annotation = torch.tensor([[1.1350e+02, 1.3725e+02, 1.3700e+02, 1.5975e+02, 0.0000e+00],
        [140, 82, 182, 120, 0.0000e+00],
        [1.4250e+02, 2.5000e-01, 1.7075e+02, 2.3500e+01, 0.0000e+00],
        [200, 72, 242, 110, 1.0000e+00],
        [2.8075e+02, 2.7600e+02, 3.1225e+02, 3.0150e+02, 0.0000e+00]])
annotation

tensor([[1.1350e+02, 1.3725e+02, 1.3700e+02, 1.5975e+02, 0.0000e+00],
        [1.4000e+02, 8.2000e+01, 1.8200e+02, 1.2000e+02, 0.0000e+00],
        [1.4250e+02, 2.5000e-01, 1.7075e+02, 2.3500e+01, 0.0000e+00],
        [2.0000e+02, 7.2000e+01, 2.4200e+02, 1.1000e+02, 1.0000e+00],
        [2.8075e+02, 2.7600e+02, 3.1225e+02, 3.0150e+02, 0.0000e+00]])

In [8]:
b = annotation
area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])
area

tensor([ 528.7500, 1596.0000,  656.8125, 1596.0000,  803.2500])

In [9]:
torch.unsqueeze(a[:, 2], dim=1)
# b[:, 2]

tensor([[180.],
        [188.],
        [196.],
        [204.],
        [212.],
        [220.],
        [228.],
        [236.],
        [244.],
        [292.]])

In [10]:
b[:, 2]

tensor([137.0000, 182.0000, 170.7500, 242.0000, 312.2500])

In [11]:
iw = torch.min(torch.unsqueeze(a[:, 2], dim=1), b[:, 2]) - torch.max(torch.unsqueeze(a[:, 0], 1), b[:, 0])
ih = torch.min(torch.unsqueeze(a[:, 3], dim=1), b[:, 3]) - torch.max(torch.unsqueeze(a[:, 1], 1), b[:, 1])

In [12]:
iw = torch.min(torch.unsqueeze(a[:, 2], dim=1), b[:, 2]) - torch.max(torch.unsqueeze(a[:, 0], 1), b[:, 0])

In [13]:
# torch.clamp(iw * ih, min=0)

In [14]:
iw = torch.clamp(iw, min=0)
ih = torch.clamp(ih, min=0)
iw * ih

tensor([[   0., 1024.,    0.,    0.,    0.],
        [   0.,  832.,    0.,    0.,    0.],
        [   0.,  576.,    0.,    0.,    0.],
        [   0.,  320.,    0.,  104.,    0.],
        [   0.,   64.,    0.,  312.,    0.],
        [   0.,    0.,    0.,  520.,    0.],
        [   0.,    0.,    0.,  728.,    0.],
        [   0.,    0.,    0.,  832.,    0.],
        [   0.,    0.,    0.,  780.,    0.],
        [   0.,    0.,    0.,    0.,    0.]])

In [15]:
torch.clamp(iw * ih / area, min=0, max=1.0)

tensor([[0.0000, 0.6416, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5213, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3609, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2005, 0.0000, 0.0652, 0.0000],
        [0.0000, 0.0401, 0.0000, 0.1955, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3258, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4561, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5213, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4887, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [16]:
torch.unsqueeze((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), dim=1) + area - iw * ih

tensor([[1552.7500, 1596.0000, 1680.8125, 2620.0000, 1827.2500],
        [1552.7500, 1788.0000, 1680.8125, 2620.0000, 1827.2500],
        [1552.7500, 2044.0000, 1680.8125, 2620.0000, 1827.2500],
        [1552.7500, 2300.0000, 1680.8125, 2516.0000, 1827.2500],
        [1552.7500, 2556.0000, 1680.8125, 2308.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 2100.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 1892.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 1788.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 1840.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 2620.0000, 1827.2500]])

In [17]:
torch.unsqueeze((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), dim=1) + area - iw * ih

tensor([[1552.7500, 1596.0000, 1680.8125, 2620.0000, 1827.2500],
        [1552.7500, 1788.0000, 1680.8125, 2620.0000, 1827.2500],
        [1552.7500, 2044.0000, 1680.8125, 2620.0000, 1827.2500],
        [1552.7500, 2300.0000, 1680.8125, 2516.0000, 1827.2500],
        [1552.7500, 2556.0000, 1680.8125, 2308.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 2100.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 1892.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 1788.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 1840.0000, 1827.2500],
        [1552.7500, 2620.0000, 1680.8125, 2620.0000, 1827.2500]])

In [18]:
#     area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

# #     print(torch.min(torch.unsqueeze(a[:, 2], dim=1), b[:, 2]))
# #     print(torch.max(torch.unsqueeze(a[:, 0], 1), b[:, 0]))
#     iw = torch.min(torch.unsqueeze(a[:, 2], dim=1), b[:, 2]) - torch.max(torch.unsqueeze(a[:, 0], 1), b[:, 0])
#     ih = torch.min(torch.unsqueeze(a[:, 3], dim=1), b[:, 3]) - torch.max(torch.unsqueeze(a[:, 1], 1), b[:, 1])

#     iw = torch.clamp(iw, min=0)
#     ih = torch.clamp(ih, min=0)

#     ua = torch.unsqueeze((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), dim=1) + area - iw * ih

#     ua = torch.clamp(ua, min=1e-8)

#     intersection = iw * ih

#     IoU = intersection / ua

In [19]:
IoU = calc_iou(myan[0, :,:], annotation[:, :4])
IoU


tensor([[0.0000, 0.6416, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4653, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2818, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1391, 0.0000, 0.0413, 0.0000],
        [0.0000, 0.0250, 0.0000, 0.1352, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2476, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3848, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4653, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4239, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [26]:
IoU = paps_calc_iou(myan[0, :,:], annotation[:, :4])
IoU

tensor([[0.0000, 0.8208, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6389, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4222, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2258, 0.0000, 0.0714, 0.0000],
        [0.0000, 0.0438, 0.0000, 0.2199, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3777, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5479, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.6389, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5928, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [27]:
 IoU_max, IoU_argmax = torch.max(IoU, dim=1) # num_anchors x 1

In [69]:
IoU_max += 1
IoU_max

tensor([1.8208, 1.6389, 1.4222, 1.2258, 1.2199, 1.3777, 1.5479, 1.6389, 1.5928,
        1.0000])

In [29]:
IoU_argmax

tensor([1, 1, 1, 1, 3, 3, 3, 3, 3, 0])

In [30]:
targets = torch.ones([10,2]) * -1
targets

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.]])

In [33]:
iou_threshold = 0.4
torch.lt(IoU_max, 0.3)

tensor([False, False, False,  True,  True, False, False, False, False,  True])

In [35]:
torch.lt(IoU_max, iou_threshold)

tensor([False, False, False,  True,  True,  True, False, False, False,  True])

In [34]:
targets[torch.lt(IoU_max, iou_threshold), :]

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.]])

In [37]:
iou_threshold = 0.3
targets[torch.lt(IoU_max, iou_threshold), :] = 0
targets

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [ 0.,  0.],
        [ 0.,  0.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [ 0.,  0.]])

In [38]:
positive_indices = torch.ge(IoU_max, 0.4)
positive_indices

tensor([ True,  True,  True, False, False, False,  True,  True,  True, False])

In [39]:
num_positive_anchors = positive_indices.sum()
num_positive_anchors

tensor(6)

In [40]:
assigned_annotations = annotation[IoU_argmax, :]
assigned_annotations

tensor([[140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [113.5000, 137.2500, 137.0000, 159.7500,   0.0000]])

In [41]:
targets[positive_indices, :] = 0
targets

tensor([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [-1., -1.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])

In [42]:
assigned_annotations[positive_indices, 4].long()

tensor([0, 0, 0, 1, 1, 1])

In [43]:
targets[positive_indices,:]

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [44]:
targets[:,assigned_annotations[positive_indices, 4].long()]

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [-1., -1., -1., -1., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [45]:
targets[positive_indices, assigned_annotations[positive_indices, 4].long()]

tensor([0., 0., 0., 0., 0., 0.])

In [46]:
assigned_annotations

tensor([[140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [140.0000,  82.0000, 182.0000, 120.0000,   0.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [200.0000,  72.0000, 242.0000, 110.0000,   1.0000],
        [113.5000, 137.2500, 137.0000, 159.7500,   0.0000]])

In [47]:
positive_indices

tensor([ True,  True,  True, False, False, False,  True,  True,  True, False])

In [48]:
assigned_annotations[positive_indices, 4].long()

tensor([0, 0, 0, 1, 1, 1])

In [49]:
targets[[ True,  True, False, False, False, False,  True,  True,  True, False],:]

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [50]:
targets[[ True,  True, False, False, False, False,  True,  True,  True, False],[0, 0, 1, 1, 1]]

tensor([0., 0., 0., 0., 0.])

In [51]:
targets

tensor([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [-1., -1.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])

In [52]:
targets[positive_indices, assigned_annotations[positive_indices, 4].long()] = 1
targets

tensor([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [-1., -1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  0.]])

In [53]:
alpha = 0.25
alpha_factor = torch.ones(targets.shape) * alpha
alpha_factor

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500]])

In [54]:
alpha_factor = torch.where(torch.eq(targets, 1.), alpha_factor, 1. - alpha_factor)
alpha_factor

tensor([[0.2500, 0.7500],
        [0.2500, 0.7500],
        [0.2500, 0.7500],
        [0.7500, 0.7500],
        [0.7500, 0.7500],
        [0.7500, 0.7500],
        [0.7500, 0.2500],
        [0.7500, 0.2500],
        [0.7500, 0.2500],
        [0.7500, 0.7500]])

In [55]:
classification =torch.rand([10,2])

In [56]:
focal_weight = torch.where(torch.eq(targets, 1.), 1. - classification, classification)
focal_weight

tensor([[0.8030, 0.1335],
        [0.2506, 0.5786],
        [0.4404, 0.4075],
        [0.4682, 0.4669],
        [0.3712, 0.5615],
        [0.2450, 0.8486],
        [0.0079, 0.2464],
        [0.2813, 0.4184],
        [0.7936, 0.5294],
        [0.0708, 0.9235]])

In [57]:
gamma=2
focal_weight = alpha_factor * torch.pow(focal_weight, gamma)
focal_weight

tensor([[1.6121e-01, 1.3358e-02],
        [1.5695e-02, 2.5105e-01],
        [4.8486e-02, 1.2452e-01],
        [1.6440e-01, 1.6352e-01],
        [1.0333e-01, 2.3644e-01],
        [4.5012e-02, 5.4014e-01],
        [4.6710e-05, 1.5181e-02],
        [5.9366e-02, 4.3767e-02],
        [4.7230e-01, 7.0064e-02],
        [3.7567e-03, 6.3964e-01]])

In [58]:
bce = -(targets * torch.log(classification) + (1.0 - targets) * torch.log(1.0 - classification))
bce

tensor([[ 1.6247,  0.1432],
        [ 0.2884,  0.8641],
        [ 0.5805,  0.5234],
        [ 0.6315,  0.6291],
        [ 0.4639,  0.8243],
        [-0.8445,  3.6120],
        [ 0.0079,  0.2829],
        [ 0.3304,  0.5420],
        [ 1.5777,  0.7537],
        [ 0.0734,  2.5704]])

In [59]:
cls_loss = focal_weight * bce
cls_loss

tensor([[ 2.6192e-01,  1.9135e-03],
        [ 4.5270e-03,  2.1693e-01],
        [ 2.8147e-02,  6.5169e-02],
        [ 1.0381e-01,  1.0287e-01],
        [ 4.7932e-02,  1.9491e-01],
        [-3.8015e-02,  1.9510e+00],
        [ 3.7009e-07,  4.2952e-03],
        [ 1.9613e-02,  2.3721e-02],
        [ 7.4516e-01,  5.2810e-02],
        [ 2.7575e-04,  1.6442e+00]])

In [70]:
targets

tensor([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [-1., -1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  0.]])

In [82]:
IoU_max.unsqueeze(dim=0).transpose(1,0)

tensor([[1.8208],
        [1.6389],
        [1.4222],
        [1.2258],
        [1.2199],
        [1.3777],
        [1.5479],
        [1.6389],
        [1.5928],
        [1.0000]])

In [87]:
IoU_max -= 0.5

In [90]:
IoU_max.unsqueeze(dim=0).transpose(1,0) + 0.5

tensor([[1.8208],
        [1.6389],
        [1.4222],
        [1.2258],
        [1.2199],
        [1.3777],
        [1.5479],
        [1.6389],
        [1.5928],
        [1.0000]])

In [91]:
f_weight = targets * (IoU_max.unsqueeze(dim=0).transpose(1,0))
f_weight = torch.clamp(f_weight, min=0.25)
f_weight

tensor([[1.3208, 0.2500],
        [1.1389, 0.2500],
        [0.9222, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 0.2500],
        [0.2500, 1.0479],
        [0.2500, 1.1389],
        [0.2500, 1.0928],
        [0.2500, 0.2500]])

In [72]:
targets ** IoU_max

RuntimeError: The size of tensor a (2) must match the size of tensor b (10) at non-singleton dimension 1

In [61]:
cls_loss = torch.where(torch.ne(targets, -1.0), cls_loss, torch.zeros(cls_loss.shape))
cls_loss

tensor([[2.6192e-01, 1.9135e-03],
        [4.5270e-03, 2.1693e-01],
        [2.8147e-02, 6.5169e-02],
        [1.0381e-01, 1.0287e-01],
        [4.7932e-02, 1.9491e-01],
        [0.0000e+00, 0.0000e+00],
        [3.7009e-07, 4.2952e-03],
        [1.9613e-02, 2.3721e-02],
        [7.4516e-01, 5.2810e-02],
        [2.7575e-04, 1.6442e+00]])

In [62]:
num_positive_anchors

tensor(6)

In [63]:
cls_loss.sum()

tensor(3.5182)

In [64]:
torch.clamp(num_positive_anchors.float(), min=1.0)

tensor(6.)

In [65]:
cls_loss.sum()/torch.clamp(num_positive_anchors.float(), min=1.0)

tensor(0.5864)

In [66]:
anchor_widths  = myan[0,:, 2] - myan[0,:, 0]
anchor_heights = myan[0,:, 3] - myan[0,:, 1]
anchor_ctr_x   = myan[0,:, 0] + 0.5 * anchor_widths
anchor_ctr_y   = myan[0,:, 1] + 0.5 * anchor_heights

In [ ]:
anchor_widths

In [ ]:
assigned_annotations = assigned_annotations[positive_indices, :4]

anchor_widths_pi = anchor_widths[positive_indices]
anchor_heights_pi = anchor_heights[positive_indices]
anchor_ctr_x_pi = anchor_ctr_x[positive_indices]
anchor_ctr_y_pi = anchor_ctr_y[positive_indices]

gt_widths  = assigned_annotations[:, 2] - assigned_annotations[:, 0]
gt_heights = assigned_annotations[:, 3] - assigned_annotations[:, 1]
gt_ctr_x   = assigned_annotations[:, 0] + 0.5 * gt_widths
gt_ctr_y   = assigned_annotations[:, 1] + 0.5 * gt_heights

In [ ]:
# clip widths to 1
gt_widths  = torch.clamp(gt_widths, min=1)
gt_heights = torch.clamp(gt_heights, min=1)

targets_dx = (gt_ctr_x - anchor_ctr_x_pi) / anchor_widths_pi
targets_dy = (gt_ctr_y - anchor_ctr_y_pi) / anchor_heights_pi
targets_dw = torch.log(gt_widths / anchor_widths_pi)
targets_dh = torch.log(gt_heights / anchor_heights_pi)

targets = torch.stack((targets_dx, targets_dy, targets_dw, targets_dh))
targets = targets.t()

In [ ]:
targets

In [ ]:
targets = targets/torch.Tensor([[0.1, 0.1, 0.2, 0.2]])
targets

In [ ]:
regression = torch.rand([10, 4])

In [ ]:

targets = targets/torch.Tensor([[0.1, 0.1, 0.2, 0.2]])

negative_indices = 1 + (~positive_indices)

regression_diff = torch.abs(targets - regression[positive_indices, :])

#                 change the beta value from 1/9 to 1.0
regression_loss = torch.where(
    torch.le(regression_diff, 1.0),
    0.5 * torch.pow(regression_diff, 2),
    regression_diff - 0.5
)
regression_losses.append(regression_loss.mean())

In [ ]:
regression_loss

In [ ]:
a = torch.randn([4,6])
a

In [ ]:
a[[1,1,1,0,1],[2,3,2,4,0]]

In [ ]:
t = torch.tensor([
    [1,2,3],[4,5,6.]
])

In [ ]:
t

In [ ]:
# # 마스크 배열을 이용하여 True 값만 추출
# t[t > 3]
# : tensor([4., 5., 6.])

# # 슬라이스를 이용하여 일괄 대입
# t[:, 1] = 10

# # 마스크 배열을 사용하여 일괄 대입
# t[t > 5] = 20

In [ ]:
t[t > 3] = 1
t

In [ ]:
t[:, 1] = 10
t

In [ ]:
# print(x)
x = torch.tensor([[0.4898, 0.2505, 0.6500],
        [0.0976, 0.4117, 0.9705],
        [0.7069, 0.0546, 0.7824],
        [0.4921, 0.9863, 0.3936]])

In [ ]:
a = torch.tensor([1,3])
b = torch.tensor([1,2,1])

In [ ]:
x[np.ix_(a,b)]

In [ ]:
y = np.arange(35).reshape(5, 7)
# y = [[0, 1, 2, 3, 4, 5, 6],
#    [7, 8, 9, 10, 11, 12, 13],
#    [14, 15, 16, 17, 18, 19, 20],
#    [21, 22, 23, 24, 25, 26, 27],
#    [28, 29, 30, 31, 32, 33, 34]]
y

In [ ]:

b = y > 20
b
# print(y[b]) #[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]

In [ ]:
b[: , 5] 

In [ ]:
y[b[ : , 5]]

In [ ]:
# b[: , 5] = [False False False True True]
print(y[b[ : , 5]])
# [[21 22 23 24 25 26 27],
#  [28 29 30 31 32 33 34]]

In [ ]:
x = np.arange(30).reshape(2, 3, 5)
x


In [ ]:
f = np.arange(15).reshape(3, 5)
f

In [ ]:
b1 = np.array([True, True, False])
b2 = np.array([False, True, False, True, True])

print(f[b1, :])
# [[0 1 2 3 4]
#  [5 6 7 8 9]]


In [ ]:
print(f[:, b2])
# [[ 1  3  4]
#  [ 6  8  9]
#  [11 13 14]]

In [ ]:
f[b1, b2]

In [ ]:
x = np.array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])
x

In [ ]:
x[1:, [2,0,1]]

In [ ]:
x[np.array([[1,2]]), np.array([2,0,1])]

In [ ]:
y = np.arange(35).reshape(5,7)
y

In [ ]:
y[np.array([0,2,4]), np.array([0,1,2])] = -1
y

In [ ]:
y[[0,2,4], :]